# Importing necessary libraries


In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_squared_error
import seaborn as sns
import logging
import os

os.chdir(r'c:\Users\habteyes.asfaw\10Accadamy\Rossmann-Sales-Prediction'
)
from src.logger import get_logger

In [3]:
logger = logging.getLogger()

# Load the dataset


In [8]:
# Function to load and preprocess the data
# Load and merge data
train_df = pd.read_csv('data/train.csv', parse_dates=['Date'], index_col='Date')
store_df = pd.read_csv('data/store.csv')
test_df = pd.read_csv('data/test.csv',parse_dates=['Date'], index_col='Date')




C:\Users\habteyes.asfaw\AppData\Local\Temp\ipykernel_11684\468474486.py:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv('data/train.csv', parse_dates=['Date'], index_col='Date')


In [9]:
# Merge store data into the training set
data = pd.merge(train_df, store_df, on='Store', how='left')

In [11]:
data.columns

Index(['Sales', 'Store', 'Customers', 'Open', 'Promo', 'StateHoliday',
       'SchoolHoliday'],
      dtype='object')

In [10]:
# Select necessary columns
data = data[['Sales', 'Store', 'Customers', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday']]


# Fill missing values


In [ ]:
data['Sales'].fillna(0, inplace=True)
data.fillna(method='ffill', inplace=True)

In [13]:
# Resample data by day and take average
data = data.resample('D').mean().fillna(method='ffill')

TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'RangeIndex'